<a href="https://colab.research.google.com/github/armstrongg1/Prevendo-Campeao-Brasileirao/blob/master/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import warnings
warnings.filterwarnings('ignore')    
import matplotlib.pyplot as plt
import matplotlib
matplotlib.use("Agg") #Needed to save figures
import pandas as pd
import numpy as np

In [2]:
def clean_currency(x):
    """ If the value is a string, then remove currency symbol and delimiters
    otherwise, the value is numeric and can be converted
    """
    #print (x)
    if x is not None:
        #x.replace('€', '').replace(',', '')
        posicao_parenteses_T = str(x).find("T")   
                                
        posicao_parenteses_M = str(x).find("m")   
        
        if posicao_parenteses_T != -1: 
            #x = x[1:posicao_parenteses_T]
            #x = float(x)*1000
            x=0
                                
        if posicao_parenteses_M != -1: 
            #if x[0].isdigit():
                #x = x[0:posicao_parenteses_M-1]
                #x = float(x)*1000000
            #elif x[1].isdigit():
                #x = x[1:posicao_parenteses_M-1]
                #x = float(x)*1000000
            x =1
        #print (str(x).find('.'))
    '''    
    if str(x).find('.') == 1:
        
        x = x * 1000000
        
    elif str(x).find('.') == 2:        
        x = x * 10       
    '''
    #print(x)
    #x = round(x,0)
    #print (x)
    return(x)


In [3]:

#carrega planilha recuperada dos dados de todos os jogos de todos os jogadores em ativa
jogadores_desempenho_treino_inicial_0 = pd.read_csv('jogadores_seriaA_2020_desempenho_por_data.csv', index_col=0)

jogadores_desempenho_treino_inicial_1 = jogadores_desempenho_treino_inicial_0.drop(np.where(jogadores_desempenho_treino_inicial_0['jogador_data_jogo'] > '2019-01-01')[0])

jogadores_desempenho_teste_inicial_1 = jogadores_desempenho_treino_inicial_0.drop(np.where(jogadores_desempenho_treino_inicial_0['jogador_data_jogo'] < '2019-01-01')[0])


In [4]:
jogadores_desempenho_treino = jogadores_desempenho_treino_inicial_1.filter([
'jogador_valor_de_mercado',
'jogador_time_visitante',
'jogador_time_casa',
'jogador_time',
'jogador_rodada_jogo',
'jogador_posicao',
'jogador_nacionalidade',
'jogador_idade',
'jogador_id',
'jogador_gols_convocacoes',
'jogador_convocacoes',
'jogador_altura',
'jogador_media_de_pontos',
'jogador_media_minutos_em_campo',
'jogador_media_gols_no_jogo',
'jogador_media_assistencias_no_jogo',
'jogador_joga_casa',
'jogador_media_quantidade_cartoes_primeiro_amarelo',
'jogador_media_quantidade_cartoes_segundo_amarelo',
'jogador_media_quantidade_cartoes_vermelho',
'jogador_media_vitorias',
'jogador_vendido',
'jogador_pontos',
'jogador_time_atual_partida',
'jogador_minutos_em_campo',
'jogador_gols_no_jogo',
'jogador_assistencias_no_jogo',
'jogador_primeiro_amarelo',
'jogador_segundo_amarelo',
'jogador_vermelho',
'vencedor'
#'time_media_gols_casa',
#'time_media_gols_fora'
#'media_de_pontos_do_time',
    ])

In [5]:
jogadores_desempenho_teste = jogadores_desempenho_teste_inicial_1.filter([
'jogador_valor_de_mercado',
'jogador_time_visitante',
'jogador_time_casa',
'jogador_time',
'jogador_rodada_jogo',
'jogador_posicao',
'jogador_nacionalidade',
'jogador_idade',
'jogador_id',
'jogador_gols_convocacoes',
'jogador_convocacoes',
'jogador_altura',
'jogador_media_de_pontos',
'jogador_media_minutos_em_campo',
'jogador_media_gols_no_jogo',
'jogador_media_assistencias_no_jogo',
'jogador_joga_casa',
'jogador_media_quantidade_cartoes_primeiro_amarelo',
'jogador_media_quantidade_cartoes_segundo_amarelo',
'jogador_media_quantidade_cartoes_vermelho',
'jogador_media_vitorias',
'jogador_vendido',
'jogador_pontos',
'jogador_time_atual_partida',
'jogador_minutos_em_campo',
'jogador_gols_no_jogo',
'jogador_assistencias_no_jogo',
'jogador_primeiro_amarelo',
'jogador_segundo_amarelo',
'jogador_vermelho',
'vencedor'
#'time_media_gols_casa',
#'time_media_gols_fora'
#'media_de_pontos_do_time',
    ])

In [6]:
#se não tenho estes dados, tenho que remover as linhas, já que são os dados principais
jogadores_desempenho_treino.dropna(subset=['jogador_time_casa'],inplace=True)
jogadores_desempenho_teste.dropna(subset=['jogador_time_casa'],inplace=True)


#media na altura - transformo a virgual em ponto para fazer calculos
jogadores_desempenho_treino['jogador_altura'] = jogadores_desempenho_treino['jogador_altura'].str.replace(',', '.')
jogadores_desempenho_teste['jogador_altura'] = jogadores_desempenho_teste['jogador_altura'].str.replace(',', '.')


#arruma campo altura
jogadores_desempenho_treino['jogador_altura'] = jogadores_desempenho_treino['jogador_altura'].astype(float)
jogadores_desempenho_teste['jogador_altura'] = jogadores_desempenho_teste['jogador_altura'].astype(float)

#preenche altura faltantes com a media por posicao
jogadores_desempenho_treino['jogador_altura'].fillna(jogadores_desempenho_treino.groupby('jogador_posicao')['jogador_altura'].transform('mean'), inplace = True)
jogadores_desempenho_teste['jogador_altura'].fillna(jogadores_desempenho_teste.groupby('jogador_posicao')['jogador_altura'].transform('mean'), inplace = True)

#delete jogador sem altura..tem um só com posicao forward
jogadores_desempenho_treino.dropna(subset=['jogador_altura'],inplace=True)
jogadores_desempenho_teste.dropna(subset=['jogador_altura'],inplace=True)

#coloco zero para quem nunca foi pra seleção
jogadores_desempenho_treino['jogador_gols_convocacoes'].fillna(0, inplace = True)
jogadores_desempenho_treino['jogador_convocacoes'].fillna(0, inplace = True)

jogadores_desempenho_teste['jogador_gols_convocacoes'].fillna(0, inplace = True)
jogadores_desempenho_teste['jogador_convocacoes'].fillna(0, inplace = True)

#time joga fora ou dentro de casa
jogadores_desempenho_teste['jogador_joga_casa'] = np.where(jogadores_desempenho_teste['jogador_time_casa'].str.strip()==jogadores_desempenho_teste['jogador_time'].str.strip(), 1,0)
jogadores_desempenho_treino['jogador_joga_casa'] = np.where(jogadores_desempenho_treino['jogador_time_casa'].str.strip()==jogadores_desempenho_treino['jogador_time'].str.strip(), 1,0)

#verificando se jogador foi vendido ou não
jogadores_desempenho_treino['jogador_vendido'] = np.where((jogadores_desempenho_treino['jogador_time']!=jogadores_desempenho_treino['jogador_time_casa']) & (jogadores_desempenho_treino['jogador_time']!=jogadores_desempenho_treino['jogador_time_visitante']), 1,0)
jogadores_desempenho_teste['jogador_vendido'] = np.where((jogadores_desempenho_teste['jogador_time']!=jogadores_desempenho_teste['jogador_time_casa']) & (jogadores_desempenho_teste['jogador_time']!=jogadores_desempenho_teste['jogador_time_visitante']), 1,0)



In [7]:
jogadores_desempenho_treino['jogador_valor_de_mercado'] = jogadores_desempenho_treino['jogador_valor_de_mercado'].apply(clean_currency).astype(float)
jogadores_desempenho_teste['jogador_valor_de_mercado'] = jogadores_desempenho_teste['jogador_valor_de_mercado'].apply(clean_currency).astype(float)



#coloco preço nos jogadores de tres maneiras possíveis: primeira pegando time/posicao, segundo pegando só time, terceira pegando tudo
jogadores_desempenho_treino['jogador_valor_de_mercado'].fillna(jogadores_desempenho_treino.groupby(['jogador_time', 'jogador_posicao'])['jogador_valor_de_mercado'].transform('median'), inplace = True)
jogadores_desempenho_treino['jogador_valor_de_mercado'].fillna(jogadores_desempenho_treino.groupby('jogador_time')['jogador_valor_de_mercado'].transform('median'), inplace = True)
jogadores_desempenho_treino['jogador_valor_de_mercado'].fillna(jogadores_desempenho_treino['jogador_valor_de_mercado'].median(), inplace = True)

#coloco preço nos jogadores de tres maneiras possíveis: primeira pegando time/posicao, segundo pegando só time, terceira pegando tudo
jogadores_desempenho_teste['jogador_valor_de_mercado'].fillna(jogadores_desempenho_teste.groupby(['jogador_time', 'jogador_posicao'])['jogador_valor_de_mercado'].transform('median'), inplace = True)
jogadores_desempenho_teste['jogador_valor_de_mercado'].fillna(jogadores_desempenho_teste.groupby('jogador_time')['jogador_valor_de_mercado'].transform('median'), inplace = True)
jogadores_desempenho_teste['jogador_valor_de_mercado'].fillna(jogadores_desempenho_teste['jogador_valor_de_mercado'].median(), inplace = True)


In [8]:

#colocando a media para algumas variaveis - media
jogadores_desempenho_treino['jogador_media_de_pontos'] = jogadores_desempenho_treino.groupby('jogador_id')['jogador_pontos'].transform('mean').astype(int)
jogadores_desempenho_treino['jogador_media_minutos_em_campo'] = jogadores_desempenho_treino.groupby('jogador_id')['jogador_minutos_em_campo'].transform('mean').astype(int)
jogadores_desempenho_treino['jogador_media_gols_no_jogo'] = jogadores_desempenho_treino.groupby('jogador_id')['jogador_gols_no_jogo'].transform('mean').astype(int)
jogadores_desempenho_treino['jogador_media_assistencias_no_jogo'] = jogadores_desempenho_treino.groupby('jogador_id')['jogador_assistencias_no_jogo'].transform('mean').astype(int)
#media cartao amarelo
jogadores_desempenho_treino['jogador_primeiro_amarelo'] = np.where(jogadores_desempenho_treino['jogador_primeiro_amarelo']==-1, 0,1)
jogadores_desempenho_treino['jogador_media_quantidade_cartoes_primeiro_amarelo'] = jogadores_desempenho_treino.groupby('jogador_id')['jogador_primeiro_amarelo'].transform('mean').astype(int)
#media segundo cartao amarelo
jogadores_desempenho_treino['jogador_segundo_amarelo'] = np.where(jogadores_desempenho_treino['jogador_segundo_amarelo']==-1, 0,1)
jogadores_desempenho_treino['jogador_media_quantidade_cartoes_segundo_amarelo'] = jogadores_desempenho_treino.groupby('jogador_id')['jogador_segundo_amarelo'].transform('mean').astype(int)
#media cartao vermelho
jogadores_desempenho_treino['jogador_vermelho'] = np.where(jogadores_desempenho_treino['jogador_vermelho']==-1, 0,1)
jogadores_desempenho_treino['jogador_media_quantidade_cartoes_vermelho'] = jogadores_desempenho_treino.groupby('jogador_id')['jogador_vermelho'].transform('mean').astype(int)
#media de vitorias do jogador
jogadores_desempenho_treino['jogador_media_vitorias'] = np.where(jogadores_desempenho_treino['vencedor']==jogadores_desempenho_treino['jogador_time'], 1,0)
jogadores_desempenho_treino['jogador_media_vitorias'] = jogadores_desempenho_treino.groupby('jogador_id')['jogador_media_vitorias'].transform('mean').astype(int)


In [9]:

#pegando as medias para variavel teste
treino_agrupada_id_cheio = jogadores_desempenho_treino.filter([
    'jogador_id',
    'jogador_media_de_pontos',
'jogador_media_minutos_em_campo',
'jogador_media_gols_no_jogo',
'jogador_media_assistencias_no_jogo',
'jogador_media_quantidade_cartoes_primeiro_amarelo',
'jogador_media_quantidade_cartoes_segundo_amarelo',
'jogador_media_quantidade_cartoes_vermelho',
'jogador_media_vitorias'
    ])

df = treino_agrupada_id_cheio.drop_duplicates('jogador_id', keep='last')
df_2 = jogadores_desempenho_teste.drop_duplicates('jogador_id', keep='last')
df_2 = df_2['jogador_id']

treino_agrupada_id = pd.DataFrame()
treino_agrupada_id['jogador_id'] = np.unique(jogadores_desempenho_treino['jogador_id'])
jogadores_desempenho_teste = pd.merge(df, jogadores_desempenho_teste, on=['jogador_id'], how='right')



In [10]:

#colocando a media para algumas variaveis de jogadores que não existem no futuro, ou seja, em 2019 há novos jogadores - media
jogadores_desempenho_teste['jogador_media_de_pontos'].fillna(jogadores_desempenho_teste.groupby('jogador_posicao')['jogador_pontos'].transform('mean').astype(int), inplace = True)
jogadores_desempenho_teste['jogador_media_minutos_em_campo'].fillna(jogadores_desempenho_teste.groupby('jogador_posicao')['jogador_minutos_em_campo'].transform('mean').astype(int), inplace = True)
jogadores_desempenho_teste['jogador_media_gols_no_jogo'].fillna(jogadores_desempenho_teste.groupby('jogador_posicao')['jogador_gols_no_jogo'].transform('mean').astype(int), inplace = True)
jogadores_desempenho_teste['jogador_media_assistencias_no_jogo'].fillna(jogadores_desempenho_teste.groupby('jogador_posicao')['jogador_assistencias_no_jogo'].transform('mean').astype(int), inplace = True)
#media cartao amarelo
#jogadores_desempenho_teste['jogador_primeiro_amarelo'] = np.where(jogadores_desempenho_teste['jogador_primeiro_amarelo']==-1, 0,1)
jogadores_desempenho_teste['jogador_media_quantidade_cartoes_primeiro_amarelo'].fillna(jogadores_desempenho_teste.groupby('jogador_posicao')['jogador_media_quantidade_cartoes_primeiro_amarelo'].transform('mean').astype(int), inplace = True)
#media segundo cartao amarelo
#jogadores_desempenho_teste['jogador_segundo_amarelo'] = np.where(jogadores_desempenho_treino['jogador_segundo_amarelo']==-1, 0,1)
jogadores_desempenho_teste['jogador_media_quantidade_cartoes_segundo_amarelo'].fillna(jogadores_desempenho_teste.groupby('jogador_posicao')['jogador_media_quantidade_cartoes_segundo_amarelo'].transform('mean').astype(int), inplace = True)
#media cartao vermelho
#jogadores_desempenho_teste['jogador_vermelho'] = np.where(jogadores_desempenho_treino['jogador_vermelho']==-1, 0,1)
jogadores_desempenho_teste['jogador_media_quantidade_cartoes_vermelho'].fillna(jogadores_desempenho_teste.groupby('jogador_posicao')['jogador_media_quantidade_cartoes_vermelho'].transform('mean').astype(int), inplace = True)

jogadores_desempenho_teste['jogador_media_vitorias'].fillna(jogadores_desempenho_teste.groupby('jogador_time_atual_partida')['jogador_media_vitorias'].transform('mean').astype(int), inplace = True)
#jogadores_desempenho_teste['jogador_media_vitorias'] = jogadores_desempenho_teste.groupby('jogador_time_atual')['jogador_posicao'].transform('mean')


In [11]:

#ainda ficou um jogador sem minutos em campo
jogadores_desempenho_treino.dropna(subset=['jogador_minutos_em_campo'],inplace=True)
jogadores_desempenho_teste.dropna(subset=['jogador_minutos_em_campo'],inplace=True)

Y_treino = pd.DataFrame(jogadores_desempenho_treino.filter(['jogador_pontos']) )

In [12]:
jogadores_desempenho_treino.drop([
  #'jogador_vendido',
         'jogador_minutos_em_campo',
         #'jogador_media_minutos_em_campo',
         'jogador_gols_no_jogo',
         'jogador_assistencias_no_jogo',
         'jogador_primeiro_amarelo',
         'jogador_segundo_amarelo',
         'jogador_vermelho',
         'vencedor',
         'jogador_pontos',
         #'jogador_id',
         #'jogador_valor_de_mercado',
#'jogador_nacionalidade',
#'jogador_idade',
#'jogador_altura',
#'jogador_time_visitante',
#'jogador_time_casa',
#'jogador_time',
#'jogador_media_vitorias',
#'jogador_media_de_pontos'
         ], axis=1, inplace=True)

Y_teste = pd.DataFrame(jogadores_desempenho_teste.filter(['jogador_pontos']))

jogadores_desempenho_teste.drop([
   #'jogador_vendido',
         'jogador_minutos_em_campo',
         #'jogador_media_minutos_em_campo',
         'jogador_gols_no_jogo',
         'jogador_assistencias_no_jogo',
         'jogador_primeiro_amarelo',
         'jogador_segundo_amarelo',
         'jogador_vermelho',
         'vencedor',
         'jogador_pontos',
         #'jogador_id',
         #'jogador_valor_de_mercado',
#'jogador_nacionalidade',
#'jogador_idade',
#'jogador_altura',
#'jogador_time_visitante',
#'jogador_time_casa',
#'jogador_time',
#'jogador_media_vitorias',
#'jogador_media_de_pontos'
         ], axis=1, inplace=True)

In [13]:
jogadores_desempenho_teste['jogador_media_de_pontos'] = jogadores_desempenho_teste['jogador_media_de_pontos'].astype(int)
jogadores_desempenho_teste['jogador_media_vitorias'] = jogadores_desempenho_teste['jogador_media_vitorias'].astype(int)
jogadores_desempenho_teste['jogador_media_quantidade_cartoes_vermelho'] = jogadores_desempenho_teste['jogador_media_quantidade_cartoes_vermelho'].astype(int)
jogadores_desempenho_teste['jogador_media_quantidade_cartoes_segundo_amarelo'] = jogadores_desempenho_teste['jogador_media_quantidade_cartoes_segundo_amarelo'].astype(int)
jogadores_desempenho_teste['jogador_media_quantidade_cartoes_primeiro_amarelo'] = jogadores_desempenho_teste['jogador_media_quantidade_cartoes_primeiro_amarelo'].astype(int)
jogadores_desempenho_teste['jogador_media_assistencias_no_jogo'] = jogadores_desempenho_teste['jogador_media_assistencias_no_jogo'].astype(int)
#jogadores_desempenho_teste['jogador_media_minutos_em_campo'] = jogadores_desempenho_teste['jogador_media_minutos_em_campo'].astype(int)
jogadores_desempenho_teste['jogador_media_gols_no_jogo'] = jogadores_desempenho_teste['jogador_media_gols_no_jogo'].astype(int)

#ainda ficou jogador sem time
#jogadores_desempenho_treino['jogador_time'].fillna('Sem time',inplace=True)

#verifica se tem alguma coisa faltando para treino
missing_data_summary_treino2 = pd.DataFrame({'columns': jogadores_desempenho_treino.columns, 'types': jogadores_desempenho_treino.dtypes, 
                                     'missing(%)': jogadores_desempenho_treino.isnull().mean().round(4) * 100,   'unicos' : jogadores_desempenho_treino.nunique()})
missing_data_summary_treino2.sort_values(by=['missing(%)'], inplace=True)

#verifica se tem alguma coisa faltando para teste
missing_data_summary_teste2 = pd.DataFrame({'columns': jogadores_desempenho_teste.columns, 'types': jogadores_desempenho_teste.dtypes, 
                                     'missing(%)': jogadores_desempenho_teste.isnull().mean().round(4) * 100,   'unicos' : jogadores_desempenho_teste.nunique()})
missing_data_summary_teste2.sort_values(by=['missing(%)'], inplace=True)

jogadores_filtrados_treino = jogadores_desempenho_treino 

jogadores_filtrados_teste = jogadores_desempenho_teste

#validacao_1 = jogadores_filtrados_treino.groupby(['jogador_time_atual_partida','jogador_rodada_jogo']).agg({'jogador_time_atual_partida': 'count'}).rename(columns={'jogador_time_atual_partida':'jogador_time_atual_partida_count'}).reset_index()

#passa tudo pra numerico (o que ainda não é)
jogadores_filtrados_2_treino = pd.get_dummies(jogadores_filtrados_treino)
jogadores_filtrados_2_teste = pd.get_dummies(jogadores_filtrados_teste)

In [14]:

#coloca na variavel para depois treinar o modelo
X_treino = jogadores_filtrados_2_treino
X_teste = jogadores_filtrados_2_teste

In [15]:
#alinhar as colunas de teste e treino - isso pq algumas colunas na base de treino são criadas a mais ja que tem mais dados. Mas não sera utilizada na base de teste
X_treino_2,X_teste_2 = X_treino.align(X_teste, join='left', axis=1, fill_value=0)


In [16]:

from sklearn.feature_selection import chi2,f_classif,mutual_info_classif,f_regression,SelectKBest,SelectFromModel
from sklearn.metrics import accuracy_score,mean_absolute_error,f1_score
from sklearn.linear_model import LinearRegression, Ridge, Lasso,LogisticRegression
from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier
#from xgboost.sklearn import XGBClassifier  
import xgboost as xgb
from xgboost.sklearn import XGBClassifier  
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import log_loss
import tensorflow as tf


from sklearn.model_selection import train_test_split

In [17]:
X_train, X_valid, y_train, y_valid = train_test_split(X_treino_2,Y_treino,test_size=0.3,random_state=1)




#y_pred_final = pd.DataFrame()
p_2 = pd.DataFrame()
p = pd.DataFrame()
result3 = pd.DataFrame() 

#k_vs_score = []
k=X_treino_2.shape[1]

In [18]:
print('  -----------------------------------------------------------')
print('  -----------------------------------------------------------')
print (k)

  -----------------------------------------------------------
  -----------------------------------------------------------
793


In [ ]:
#outro modelo

from sklearn.feature_selection import RFECV
from sklearn.model_selection import StratifiedKFold
#from sklearn.feature_selection import RFEdata = load_iris()
classifier = xgb.XGBClassifier(objective="multi:softprob",seed=0)  
selector = RFECV(estimator=classifier, step=0.1, cv=StratifiedKFold(10)) 
selector.fit(X_train,  y_train)
#print('Optimal number of features: {}'.format(rfecv.n_features_))
print('The optimal number of features is {}'.format(selector.n_features_))
features = [f for f,s in zip(X_train.columns, selector.support_) if s]
print('The selected features are:')
print ('{}'.format(features))

In [ ]:

plt.figure()
plt.xlabel("Number of features selected")
plt.ylabel("Cross validation score (roc auc)")
plt.plot(range(1, len(selector.grid_scores_) + 1), selector.grid_scores_)
plt.savefig('feature_auc_nselected.png', bbox_inches='tight', pad_inches=1)

In [ ]:

X_train_s = X_treino_2[features]

classifier.fit(X_train_s, Y_treino)

X_test_s = X_teste_2[features]
print ('Out of sample auc: {}'.format(classifier.score(X_test_s, Y_teste)))

X_sel = X_valid[features]

  

In [ ]:
#outro modelo
clf = xgb.XGBClassifier(objective="multi:softprob",seed=0)

#ratio = float(np.sum(Y_treino == 1)) / np.sum(Y_treino==0)
#clf = xgb.XGBClassifier(missing=9999999999,
 #               max_depth = 7,
  #              n_estimators=1000,
   #             learning_rate=0.1, 
    #            nthread=4,
     #           subsample=1.0,
      #          colsample_bytree=0.5,
       #         min_child_weight = 3,
        #        scale_pos_weight = ratio,
         #       seed=1301)

clf.fit(X_train_s,  y_valid, early_stopping_rounds=50, eval_metric="mlogloss",   eval_set=[(X_train_s, Y_treino), (X_test_s, Y_teste)])
sel_test = X_teste_2[features]
p['pontos_previsao'] = clf.predict(sel_test, ntree_limit=clf.best_iteration)
print ('XGBClassifier')
print (f1_score(Y_teste,p,average='weighted'))